# 마켓컬리 인기브랜드 크롤러 개발하기
---
### 1. 필수 라이브러리 설치하기
#### (1) request 라이브러리 설치
- Anaconda Cloud의 작은 브라우저 역할을 하게됨
- 특정 웹사이트에서 html에서 소스코드를 그대로 가져와서 반환해주는 역할을 하게됨
- `conda install -c anaconda requests`

#### (2) BeautifulSoup 라이브러리 설치
- html 페이지에서 원하는 형태를 찾는 것에 도움이 됨
- `conda install -c anaconda beautifulsoup4`

#### (3) tqdm 라이브러리 설치
- 작업에 대한 상태를 확인할 때 유용
- `conda install -c conda-forge tqdm`

#### 윈도우에서 conda 라이브러리를 통해 위 항목을 설치하도록 한다.
- 아나콘다 prompt를 열어주는데, 설치할 때 관리자권한으로 설치하는 것을 권장
- Anaconda Prompt를 관리자 권한으로 실행해야함
- 새로운 가상환경을 만들지 않았다면 (base)라는 문구가 터미널에서 보여야 정상

### 2. 수집할 사이트 확인하기
---
- 사이트 주소: [컬리 > 베스트 > 판매순](https://www.kurly.com/collections/market-best?page=1&per_page=96&sorted_type=1)

![kurly_best](https://ifh.cc/g/PdNT9g.jpg)

In [61]:
#주요 라이브러리 호출하기

import pandas as pd
from bs4 import BeautifulSoup as bs
import time
from tqdm import tqdm

In [62]:
#수집할 사이트 정보 가져오기

app_name = "마켓컬리"
url = "https://www.kurly.com/collections/market-best?page=1&per_page=96&sorted_type=1"

#table = pd.read_html(url)
#실행결과 테이블이 없다는 것을 확인

<b>이런 경우 `Requests`를 활용하게 됨 → 이를 활용하면 이미지, CSS파일 적용없이 소스코드만을 가져오게 됨
- 우리가 수집하려는 페이지가 `get` 방식이기 때문에, 아래에서도 `requests.get`을 활용한다.
- 실행했을 때 `<Response [200]>`이라는 결과값이 나오는데, 이는 `Headers`의 `Status Code`의 200과 동일한 의미로 잘 받았다는 의미다.


이를 그냥 활용하기 어렵기 떄문에, 우리는 `beautifulsoup` 의 도움을 받게 된다.

In [63]:
import requests

response = requests.get(url)
response.status_code

200

In [64]:
from bs4 import BeautifulSoup as bs

# Warning 메시지가 뜨는 경우, lxml을 지정해주면 사라짐
# BeautifulSoup을 파싱할 때 lxml로 파싱해달라고 지정하는 옵션으로 보면됨
html = bs(response.text, "lxml")

- 확인한 결과, 보안으로 `Beautifulsoup`을 통해 소스코드에서 상품명을 가져오는 것이 불가능한 상황
- 따라서 이 경우에는 `Selenium`을 활용해서 작업할 수 있도록 한다.

#### Selenium 설치하기
- `selenium`과 `webdriver_manager`를 설치해준다.

In [65]:
import selenium
selenium.__version__

'4.8.3'

In [66]:
import webdriver_manager
webdriver_manager.__version__

'3.8.5'

In [67]:
from selenium import webdriver

driver = webdriver.Chrome('C:/Users/win10_original/Desktop/chrome_driver/chromedriver')
driver.implicitly_wait(10)
driver.get(url)

C:\Users\win10_original\AppData\Local\Temp\ipykernel_21060\128688385.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/win10_original/Desktop/chrome_driver/chromedriver')


In [68]:
from selenium.webdriver.common.by import By

html_comments = driver.find_elements(By.CSS_SELECTOR, ".css-c1cgl.e1c07x489") #이거는 실패
element = driver.find_element(By.XPATH, '//*[@id="container"]/div/div[2]/div[2]/a[1]/div[2]/span[2]')
print(element.text)

[연세우유 x 마켓컬리] 전용목장우유 900mL


In [69]:
# 수집한 데이터를 저장할 리스트를 생성합니다.
elements = []
ranks = []

# 초기 페이지를 설정합니다.
page_num = 1

# 페이지를 순차적으로 이동하며 element를 수집합니다.
while True:
    url = f"https://www.kurly.com/collections/market-best?page={page_num}&per_page=96&sorted_type=1"
    driver.get(url)
    try:
        for i in range(1, 97):
            rank = (page_num - 1) * 96 + i
            xpath = f'//*[@id="container"]/div/div[2]/div[2]/a[{i}]/div[2]/span[2]'
            element = driver.find_element(By.XPATH, xpath)
            elements.append(element.text)
            ranks.append(rank)
    except:
        break
    page_num += 1

# 데이터프레임을 생성합니다.
df = pd.DataFrame({'Rank': ranks, 'Item': elements})

In [81]:
df

,Rank,Item,brand_KR,app
0,1,[연세우유 x 마켓컬리] 전용목장우유 900mL,연세우유,마켓컬리
1,2,DOLE 실속 바나나 1kg (필리핀),자체PB상품,마켓컬리
2,3,[스윗밸런스] 오늘의 샐러드 10종 (리뉴얼) (택1),스윗밸런스,마켓컬리
3,4,애호박 1개,자체PB상품,마켓컬리
4,5,유명산지 설향딸기 500g,자체PB상품,마켓컬리
...,...,...,...,...
280,281,[신선설농탕] 고기 설렁탕,신선설농탕,마켓컬리
281,282,3겹 천연펄프 화장지 (27m X 30롤),자체PB상품,마켓컬리
282,283,[일상味소] 채끝 250g (냉장),일상味소,마켓컬리
283,284,[발린느] 카이막 100g,발린느,마켓컬리


### 데이터 전처리 1차

- 대괄호 안에 있는 브랜드명 추출하기
    - 추출한 결과, 일반적으로 대괄호가 없거나 브랜드명 대신 KF365가 입력된 사례가 50개 이상 존재 (4월 기준)
    - 해당 항목을 전처리하는 작업이 필요할 것으로 보여짐


In [82]:
import re

# KF365 제거하기
df['Item'] = df['Item'].str.replace('\[KF365\]', '', regex=True)

#대괄호 안의 전체내용 추출하기
df['brand_KR'] = df['Item'].apply(lambda x: re.search(r'\[(.*?)\]', x).group(0) if re.search(r'\[(.*?)\]', x) else '')
df['brand_KR'] = df['brand_KR'].apply(lambda x: x[1:-1] if x else '자체PB상품')

kurly_df = df

In [83]:
print(df['brand_KR'].unique())

['연세우유 x 마켓컬리' '자체PB상품' '스윗밸런스' '사미헌' '홍대주꾸미' "Kurly's" '금미옥' '이연복의 목란'
 '전주 베테랑' '더플랜' '그릭데이' '멕시카나' '픽어베이글' '우주' '브룩클린688' '서울우유' '오늘식빵' '마이하노이'
 '햇반/쿡반' '샐러드판다' '오뗄블랙라벨' '하림' '숭의가든' '홍루이젠' '제주 삼다수' '연안식당' '모노키친' '스윗볼'
 '교토마블' '부침명장' '풀무원' '강남면옥' '창억' '자연실록' '시골보쌈과 감자옹심이' '농심' '스타벅스' '아티제'
 '포비베이글' '에머이' '폰타나' '밀클레버' '한팟' '일상味소' '라라스윗' '탄단지' '온더고' 'KS365'
 '남향푸드또띠아' '이비에' '만전김' '투다리' '최현석의 쵸이닷' '햇반' '그녀의빵공장' '떡미당' '일일특가'
 '그래놀라 하우스' '오모가리' '태우한우' '거대곰탕' '바른식' '라엘' '미자언니네' '델리치오' '도리깨침' '스위프리'
 '서울마님' '오마뎅' '석관동 떡볶이' '통뼈' '해통령' '하남주꾸미' 'YOZM' '앤쿡' '김구원선생' '쌜모네키친'
 '존쿡 델리미트' '워커힐' '한끼통살' '경복궁 BLACK' '청양' '덴프스' '켄트' 'Dole' '데체코' '동원참치'
 '채선당' '수산곳간' '도스타코스' '샹달프' '올면' '모현상회' '이즈니' '미트클레버' '크리스피도넛'
 "kim's butcher" '프로그' '마이노멀' '진실된손맛' '비움반찬' '은하수산' '착한떡' '복음자리' '감자밭'
 '네이쳐패스' '플라하반' '파세오' '후이펑' '에이원식품' '프레시지' '르네디종' '타쿠미야' '마이셰프' '광화문 미진'
 '온더바디' '외할머니댁' '벽제갈비' '크놀라' '속초해품' '풀무원다논' '하코야' '마이비밀' '덴티스테' '태극당'
 '콜린스그린' '댄케이크' '창화당' '아로마티카' '멘야하나비' '하루한킷' '폴 바셋' '리틀넥' '레인코스트'
 "Kim's Butche

In [84]:
#자체PB상품군 리스트 확인
kurly_df[kurly_df['brand_KR']=='자체PB상품']

,Rank,Item,brand_KR,app
1,2,DOLE 실속 바나나 1kg (필리핀),자체PB상품,마켓컬리
3,4,애호박 1개,자체PB상품,마켓컬리
4,5,유명산지 설향딸기 500g,자체PB상품,마켓컬리
6,7,아보카도 (1개),자체PB상품,마켓컬리
9,10,감자 1kg,자체PB상품,마켓컬리
16,17,방울토마토 500g,자체PB상품,마켓컬리
17,18,다다기오이 3입,자체PB상품,마켓컬리
18,19,양념 소불고기 1kg (냉장),자체PB상품,마켓컬리
21,22,양파 1.5kg,자체PB상품,마켓컬리
27,28,친환경 설향 딸기 500g,자체PB상품,마켓컬리


### 수집한 데이터를 하나의 데이터 프레임으로 합치기

- `pd.concat` 기능을 활용해서 하나로 합칠 수 있음
- (주의) 기간이 긴 데이터를 수집할 때는 서버에 부담을 주지 않기 위해 time.sleep() 값을 주게 된다.

In [85]:
#파생변수 생성하기
kurly_df['app'] = app_name

#브랜드명에서 공백제거하기
kurly_df['brand_KR'] = kurly_df['brand_KR'].str.replace(' ','')

cols = ['app', 'Rank', 'brand_KR']
kurly_df = kurly_df[cols]
kurly_df

,app,Rank,brand_KR
0,마켓컬리,1,연세우유x마켓컬리
1,마켓컬리,2,자체PB상품
2,마켓컬리,3,스윗밸런스
3,마켓컬리,4,자체PB상품
4,마켓컬리,5,자체PB상품
...,...,...,...
280,마켓컬리,281,신선설농탕
281,마켓컬리,282,자체PB상품
282,마켓컬리,283,일상味소
283,마켓컬리,284,발린느


In [86]:
brand_list = pd.read_csv('brand_hosting_archive.csv')

#수집된 데이터에 LEFT JOIN
merged_df = pd.merge(kurly_df, brand_list, how='left', left_on='brand_KR', right_on='brand_KR')
#merged_df = merged_df.drop(['brand_EN', 'brand_KR'], axis=1)

merged_df

,app,Rank,brand_KR,brand_EN,URL
0,마켓컬리,1,연세우유x마켓컬리,NaN,www.yonseidairy.com
1,마켓컬리,2,자체PB상품,NaN,NaN
2,마켓컬리,3,스윗밸런스,SWEETBALANCE,sweetbalance.kr
3,마켓컬리,4,자체PB상품,NaN,NaN
4,마켓컬리,5,자체PB상품,NaN,NaN
...,...,...,...,...,...
305,마켓컬리,281,신선설농탕,SINSUNSEOLNONGTANG,sinsunseolnongtang.co.kr
306,마켓컬리,282,자체PB상품,NaN,NaN
307,마켓컬리,283,일상味소,NaN,NaN
308,마켓컬리,284,발린느,NaN,NaN


### 데이터 전처리하기 및 EDA

- 중복 입력된 브랜드 필터링
- 사이트 주소 없는 브랜드 확인하기

In [87]:
import datetime

# 오늘 날짜 구하기
today = datetime.date.today()

# YYYY-MM-DD 형태의 문자열로 변환
today_str = today.strftime('%Y-%m-%d')

In [88]:
file_name = f"{app_name}_{today_str}.csv"
file_name

'마켓컬리_2023-04-05.csv'

In [89]:
merged_df.to_csv(file_name, index=False)
pd.read_csv(file_name)

,app,Rank,brand_KR,brand_EN,URL
0,마켓컬리,1,연세우유x마켓컬리,NaN,www.yonseidairy.com
1,마켓컬리,2,자체PB상품,NaN,NaN
2,마켓컬리,3,스윗밸런스,SWEETBALANCE,sweetbalance.kr
3,마켓컬리,4,자체PB상품,NaN,NaN
4,마켓컬리,5,자체PB상품,NaN,NaN
...,...,...,...,...,...
305,마켓컬리,281,신선설농탕,SINSUNSEOLNONGTANG,sinsunseolnongtang.co.kr
306,마켓컬리,282,자체PB상품,NaN,NaN
307,마켓컬리,283,일상味소,NaN,NaN
308,마켓컬리,284,발린느,NaN,NaN
